In [ ]:
import os
from jobqueue_features import (
    MPIEXEC,
    CustomSLURMCluster,
)

from jobqueue_features import on_cluster, mpi_task, get_task_mpi_comm

common_kwargs = {
    "walltime": "00:04:00",
    "cores_per_node": 2,
    "minimum_cores": 2,
    "hyperthreading_factor": 1,
    "ntasks_per_node": 2,
    "memory": "512 MB",
    "mpi_mode": True,
    "env_extra": [
        "export OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1",
        "export OMPI_ALLOW_RUN_AS_ROOT=1",
    ],
    "mpi_launcher": MPIEXEC,
    "local_directory": "/tmp",
    "queue": "batch",
}

In [ ]:
custom_cluster = CustomSLURMCluster(
    name="mpiMultiCluster",
    nodes=1,
    maximum_jobs=2,
    **common_kwargs
)

In [ ]:
@mpi_task(cluster_id=custom_cluster.name)
def lammps_task(input_file, run_steps=100):
    from mpi4py import MPI
    from lammps import PyLammps
    
    L = PyLammps()      # Initialise LAMMPS
    L.file(input_file)  # Read the input file
    L.run(run_steps)    # Simulate the system

    if MPI.COMM_WORLD.Get_rank()==0:
        return "Potential energy: %s" % L.eval("pe")
    
@on_cluster(cluster=custom_cluster)
def my_lammps_job(input_file="in.melt", run_steps=100):
    return lammps_task(input_file, run_steps=run_steps)

In [ ]:
future = my_lammps_job(run_steps=2000)
print(future.result())
future.cancel()